# NVP para ajustar datos originales y latentes

Se implementa una red neuronal conocida como flujo normalizador que ajustara en los datos originales y latentes una distribucion de probabilidad para modelar la complejidad de ambos espacios de alta dimensión.

In [7]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow_probability as tfp

In [12]:
#preprocesamiento de datos
data = pd.read_csv('Liver_GSE14520_U133A.csv')
data.drop(['samples','type'], axis=1, inplace=True)
data.head()

,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
0,6.801198,4.553189,6.787790,5.430893,3.250222,6.272688,3.413405,3.374910,3.654116,3.804983,...,10.735084,10.398843,12.298551,12.270505,3.855588,3.148321,3.366087,3.199008,3.160388,3.366417
1,7.585956,4.193540,3.763183,6.003593,3.309387,6.291927,3.754777,3.587603,5.137159,8.622475,...,11.528447,11.369919,12.867048,12.560433,4.016561,3.282867,3.541994,3.548680,3.460083,3.423348
2,7.803370,4.134075,3.433113,5.395057,3.476944,5.825713,3.505036,3.687333,4.515175,12.681439,...,10.892460,10.416151,12.356337,11.888482,3.839367,3.598851,3.516791,3.484089,3.282626,3.512024
3,6.920840,4.000651,3.754500,5.645297,3.387530,6.470458,3.629249,3.577534,5.192624,11.759412,...,10.686871,10.524836,12.006596,11.846195,3.867602,3.180472,3.309547,3.425501,3.166613,3.377499
4,6.556480,4.599010,4.066155,6.344537,3.372081,5.439280,3.762213,3.440714,4.961625,10.318552,...,11.014454,10.775566,12.657182,12.573076,4.091440,3.306729,3.493704,3.205771,3.378567,3.392938


In [13]:
# Normalización de los datos
data = data.values.astype(np.float32)
data = StandardScaler().fit_transform(data)

In [ ]:
#definimos una funcion que tomara un numero aleatorio de caracteristicas por experimento
def get_random_subset(data, n_features=1000, seed=None):
    if seed is not None:
        np.random.seed(seed)
    selected = np.random.choice(data.columns, size=n_features, replace=False)
    return data[selected]

In [14]:
tfd = tfp.distributions
tfpl = tfp.layers
tfb = tfp.bijectors

num_blocks = 3
hidden_units = 50

class RealNVP(keras.Model):
    def __init__(self, input_dim, num_blocks, hidden_units, base_distribution=None, **kwargs):
        super(RealNVP, self).__init__(**kwargs)
        self.input_dim = input_dim                                                
        self.num_blocks = num_blocks
        self.hidden_units = hidden_units
        self.base_distribution = base_distribution or tfd.MultivariateNormalDiag(  #distribucion normal diagonal de media cero y varianza uno
            loc=tf.zeros(input_dim),
            scale_diag=tf.ones(input_dim)
        )
        self.nll_tracker = keras.metrics.Mean(name="nll")
        # Flujo de z->x cadena de bijectors
        bijectors = []
        self.nets = []
        

        for i in range(num_blocks):
            net =  tfb.real_nvp_default_template(        #usamos la plantilla de red densa pequeña 
                [hidden_units, hidden_units],
                name=f"NN_{i}"
            )
            self.nets.append(net)
            bijectors.append(
                tfb.RealNVP(  
                    shift_and_log_scale_fn=net, 
                    num_masked=input_dim // 2,           #enmascara la mitad de las 20000 características
                    name=f"RealNVP_{i}"
                )
            )
            bijectors.append(                            
                tfb.Permute(
                    permutation=tf.random.shuffle(tf.range(input_dim)),  #se añade una permutación para asegurar que todas las dimensiones interactuen 
                    name=f"Permute_{i}"
                )
            )

        self.flow_bijector = tfb.Chain(list(reversed(bijectors)))
        self.flow_distribution = tfd.TransformedDistribution(            #creamos la distribucion transformada
            distribution=self.base_distribution,
            bijector=self.flow_bijector
        )
        self.trainable_vars = []                                       #los bijectors con el atributo shift_and_log_scale tienen los
        for b in self.flow_distribution.bijector.bijectors:            #parámetros de traslación y escala para las dimensiones transformadas
            if hasattr(b, "shift_and_log_scale_fn") and hasattr(b.shift_and_log_scale_fn, "trainable_variables"):
                self.trainable_vars += b.shift_and_log_scale_fn.trainable_variables


    def call(self, inputs):
            return self.flow_distribution(inputs)  
   
    @property
    def metrics(self):
            return [self.nll_tracker]

    def train_step(self, data):
            with tf.GradientTape() as tape:
                nll = -tf.reduce_mean(self.flow_distribution.log_prob(data)) #verosimilitud de que los datos originales sean observados bajo la distro 
            grads = tape.gradient(nll, self.trainable_vars)
            self.optimizer.apply_gradients(zip(grads, self.trainable_vars))
            self.nll_tracker.update_state(nll)
            return {"loss": nll}
        
                        